In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from model import MusicGen
from dataset import ShardedDataset
from parse_config import Config
import os
from pathlib import Path

from tqdm.auto import tqdm

In [2]:
min_drum_pitch = 35
max_drum_pitch = 81
n_drum_pitches = max_drum_pitch - min_drum_pitch + 1

min_pitch = 0
max_pitch = 127
n_pitches = max_pitch - min_pitch + 1

sequence_length = Config.get('design')['sequence_length']
n_velocities = 128
n_instruments = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
prefix = input("Enter the name/prefix of the processed dataset")
root = os.getcwd()
shard_path = os.path.join(root, Config.get("preprocessing")['processed_data_path'], prefix)
shards = list(Path(shard_path).rglob("*.pth"))
shards.sort()
meta_path = shards.pop()
meta = torch.load(meta_path)

C:\Users\aniru\AppData\Local\Temp\ipykernel_11592\2647486288.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  meta = torch.load(meta_path)


In [ ]:
for shard in shards:
  tmp = torch.load(shard)
  print(len(tmp))
  for song in tmp:
    print(song.shape)

In [4]:
dataset = ShardedDataset(paths=shards, rel_idxs=meta['rel_idxs'])
dataloader = DataLoader(
    dataset, 
    batch_size=5, 
    shuffle=True,
    pin_memory=True
)

In [5]:
def custom_loss_fn(out, target):
    instrument_logits = out["instrument"]
    drum_pitch_logits = out["drum_pitch"]
    regular_pitch_logits = out["regular_pitch"]
    velocity_logits = out["velocity"]
    step = out["step"]
    duration = out["duration"]

    pitch_target = target[:, 0].long()
    velocity_target = target[:, 1].long()
    duration_target = target[:, 2].unsqueeze(1)
    step_target = target[:, 3].unsqueeze(1)
    instrument_target = target[:, 4].long()

    instrument_loss = F.cross_entropy(instrument_logits, instrument_target)
    is_drum = (instrument_target == 0)

    drum_pitch_loss = F.cross_entropy(drum_pitch_logits[is_drum], pitch_target[is_drum] - min_drum_pitch) if is_drum.any() else 0
    regular_pitch_loss = F.cross_entropy(regular_pitch_logits[~is_drum], pitch_target[~is_drum]) if (~is_drum).any() else 0
    pitch_loss = drum_pitch_loss + regular_pitch_loss

    # Velocity loss
    velocity_loss = F.cross_entropy(velocity_logits, velocity_target)

    # Step and duration loss (MSE)
    step_loss = F.mse_loss(step, step_target)
    duration_loss = F.mse_loss(duration, duration_target)

    # Total loss
    total_loss = instrument_loss + pitch_loss + velocity_loss + step_loss + duration_loss
    return total_loss

In [ ]:
model = MusicGen(meta)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, (sequences, targets) in tqdm(enumerate(dataloader), total=len(dataloader)):
        # batch size, 128, 5
        sequences = sequences.to(device)
        # batch size, 5
        targets = targets.to(device)
        
        optimizer.zero_grad()

        # Predicting without hidden for default hidden = None
        out, hidden = model(sequences)
        loss = custom_loss_fn(out, targets)
        loss.backward()
        optimizer.step()
        #print(loss.item())
        total_loss += loss.item()
    torch.save(model.state_dict(), f"trained_weights/model13feb-{epoch}")    
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), f"models/model3-e{num_epochs}.pth")